<a href="https://colab.research.google.com/github/mike1336git/colab_notebook/blob/main/with_js/js088_jetDSMC2D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### simulator( html + css + js ) + control( python )

In [35]:
#@title js088_jetDSMC2D / def exec_html_js() ... exec me first
#
#  Copyright(C) 2023-2024 Mitsuru Ikeuchi
#  home page: https://mike1336.web.fc2.com/index.html
#  Released under the MIT license ( https://opensource.org/licenses/MIT )
#
#  ver 0.0.0  2023.11.29 created,  last updated on 2024.03.07
#

# def exec_html_js()

import IPython
from IPython.display import display, HTML
from google.colab.output import eval_js

def exec_html_js():
  htm = HTML('''


<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8">
<title>js088_jetDSMC2D</title>
<script type="text/javascript">

// %%%%%%%%%%%%%%%%%%%%  javaScript  %%%%%%%%%%%%%%%%%%%%

'use strict';

/* --------------------
//
//  js088_jetDSMC2D
//    Copyright(C) 2018-2023 Mitsuru Ikeuchi
//    Released under the MIT license ( https://opensource.org/licenses/MIT )
//
//    ver 0.0.0  2018.01.28 created, last updated on 2018.11.30
//    ver 0.0.1  2019.01.23 v1, last updated on 2021.08.17
//    ver 0.0.2  2021.11.05 v2, last updated on 2021.11.05
//    ver 0.0.3  2023.04.28 v3, last updated on 2023.07.20
//
// -------------------- direct simulation Monte Carlo DSMC2D
//
//  direct simulation Monte Carlo DSMC2D
//
//    cell size; Lc = 0.001 (m)
//    mean free path; lambda ~ 0.002 (m)
//    No. of representative particle in the cell; Nc ~ 30 (particles)
//    maximum relative speed; Vmax ~ 1000*2 (m/s)
//    time interval; dt = 0.2*Lc/Vmax = 1.0e-7 (s)
//    cross section of the collision; sgm = 4R = 4*3.0e-10 (m)
//    number density; n = Nc*FN/Lc^2, lambda = 1/(n*sgm)
//      n = 1/(lambda*sgm) = 1/(0.002*1.2e-9) = 4.0e11 (particles/m^2)
//    represent numbers of the real molecule; FN
//      FN = n*Lc^2/Nc = 4e11*1e-6/30 = 1.3e4 --> 2.0e4 (particles)
//    the number of collisions in the cell; Mc
//      Mc = 0.5*Nc*Nc*FN*sgm*Vmax*dt/Lc^2 ~ 4(times)
//
// --------------------
*/

const jetDSMC2D = (function(){ // ====================  jetDSMC2D Module  ====================

	const g_PI = 3.141592653598793;				// Math.PI
	const g_AMU = 1.66053904e-27;				// (kg) atomic mass unit
	const g_kB = 1.380649e-23;					// (J/K) Boltzmann's constant
	const g_NsxMax = 200;						// cell array x-max
	const g_NsyMax = 100;						// cell array y-max
	const g_NNpMax = g_NsxMax*g_NsyMax*30;		// Nc=30 : Number of representative particle in the cell

	let g_sysTime = 0.0;						// (s) system time
	let g_dt = 1.0e-7;							// (s) time step
	let g_dx = 1.0e-3;							// (m) x-division
	let g_dy = 1.0e-3;							// (m) y-division
	let g_Nsx = 120;							// array x-size of field
	let g_Nsy = 80;								// array y-size of field
	let g_xMax = g_Nsx*g_dx;					// (m) array x-size of field
	let g_yMax = g_Nsy*g_dy;					// (m) array y-size of field
	let g_ffn = 20000;							// represent numbers of the real molecule
	let g_sgm = 4.0*3.0e-10;					// (m for 2D) collision cross section
	let g_NNp = g_Nsx*g_Nsy*30;					// number of particles
	let g_periodicSW = 0;						// g_periodicSW 0:non-periodic 1:periodic
	let g_adsorptionEnergy = 0.1;				// (eV) adsporption energy

	const g_xx = dim1( g_NNpMax );							// (m) x-position of 1-th particle
	const g_yy = dim1( g_NNpMax );							// (m) y-position of 1-th particle
	const g_vx = dim1( g_NNpMax );							// (m/s) x-velocity of 1-th particle
	const g_vy = dim1( g_NNpMax );							// (m/s) xy-velocity of 1-th particle
	const g_mass = dim1( g_NNpMax );						// (kg) xmass of 1-th particle
	const g_kind = dimInt1( g_NNpMax );						// kind of 1-th particle
	const g_mark = dimInt1( g_NNpMax );						// mark of 1-th particle
	const g_section = dimInt3( g_NsxMax, g_NsyMax, 200 );	// particle number in the cell[i][j]
	const g_cellAttribute = dimInt2( g_NsxMax, g_NsyMax );		// 0:free space 1:wall 2:absorption 3:adsorption

	const g_meanN = dim2( g_NsxMax, g_NsyMax );
	const g_density = dim2( g_NsxMax, g_NsyMax );			// density of the cell[i][j]
	const g_meanDensity = dim2( g_NsxMax, g_NsyMax );
	const g_xFlow = dim2( g_NsxMax, g_NsyMax );				// x-flow velocity of the cell[i][j]
	const g_yFlow = dim2( g_NsxMax, g_NsyMax );				// y-flow velocity of the cell[i][j]
	const g_meanxFlow = dim2( g_NsxMax, g_NsyMax );			// mean x-flow velocity of the cell[i][j]
	const g_meanyFlow = dim2( g_NsxMax, g_NsyMax );			// mean y-flow velocity of the cell[i][j]
	const g_temperature = dim2( g_NsxMax, g_NsyMax );		// temperature of the cell[i][j]
	const g_pressure = dim2( g_NsxMax, g_NsyMax );			// pressure of the cell[i][j]

	const g_vDistribution = dimInt1( 200 );					// velocity distribution function
	const g_vSpace = dimInt2( 250, 250 );					// velocity-space


	function dim1( n ) {
		return new Float64Array( n );
	}

	function dimInt1( n ) {
		return new Int32Array( n );
	}

	function dim2( ni, nj ) {
		let a = [];
		for (let i=0; i<ni; i++) {
			a[i] = new Float64Array( nj );
		}
		return a;
	}

	function dimInt2( ni, nj ) {
		let a = [];
		for (let i=0; i<ni; i++) {
			a[i] = new Int32Array( nj );
		}
		return a;
	}

	function dimInt3( ni, nj, nk ) {
		let a = [];
		for (let i=0; i<ni; i++) {
			a[i] = [];
			for (let j=0; j<nj; j++) {
				a[i][j] = new Int32Array( nk );
			}
		}
		return a;
	}


	// --------------------  set initial condition  --------------------

	function setInitialCondition( theme, boundary ) {
		g_sysTime = 0.0;
		resetField();
		g_periodicSW = boundary;
		setCellandParticle();
	}

	function setCellandParticle() {
		const nsx=g_Nsx, nsy=g_Nsy, nnp=g_NNp;

		for (let i=0; i<nsx; i++) {
			for (let j=0; j<nsy; j++) {
				g_cellAttribute[i][j] = 0; // free space
				if ((i==nsx/2 || i==nsx/2+1 ) && (j<nsy/2-2 || j>nsy/2+2)) {
					g_cellAttribute[i][j] = 1; // wall
				}
			}
		}

		for (let i=0; i<nnp; i++) {
			g_kind[i] = 1;
			g_mass[i] = 28.0*g_AMU;
			g_xx[i] = g_xMax*Math.random()*0.5;
			g_yy[i] = g_yMax*Math.random();
			g_vx[i] = normal_random(1200.0);
			g_vy[i] = normal_random(1200.0);
		}
	}

	function normal_random(v) {
		return v*(Math.random()+Math.random()+Math.random()+Math.random()+Math.random()+Math.random()-3.0)/3.0;
	}


	// --------------------  time evolution  --------------------

	function timeEvolution( nCalc, setFieldSW ) {
		for (let i=0; i<nCalc; i++) {
			timeStep();
		}
		if ( setFieldSW==1 ) setField();
	}

	function timeStep() {
		g_sysTime += g_dt;
		moveParticles();
		setSection();
		interaction();
	}

	// --- movement

	function moveParticles() {
		const nnp=g_NNp;
		for (let i=0; i<nnp; i++) {
			if (g_kind[i]==1) { // 1: alive 0:dead(absorbed) -1:adsorbed
				g_xx[i] += g_vx[i]*g_dt;
				g_yy[i] += g_vy[i]*g_dt;
				const ca = cellAttributeAt(g_xx[i],g_yy[i]); // 0:free space 1:wall 2:absorption 3:adsorption
				if (ca>0) {
					if (ca==1) { // wall
						wallCell(i);
					} else if (ca==2) { // absorption
						absorbCell(i);
					} else if (ca==3) { // adsorption
						adsorbCell(i);
					}
				}
			} else if (g_kind[i]<0) { // adsorbed
				if (Math.random()<1.0*Math.exp(-g_adsorptionEnergy/0.025)) {
					g_kind[i] = -g_kind[i];
					g_vx[i] = -g_vx[i]; g_vy[i] = -g_vy[i];
				}
			}
		}

		for (let i=0; i<nnp; i++) {
			if (g_kind[i]==1) {
				if (g_xx[i] < 0.0) {
					g_xx[i] = 0.0; g_vx[i] = -g_vx[i]; g_vy[i] = g_vy[i];
				}
				if (g_xx[i] > g_xMax) { // when particle collided to wall, move to near left-hand-side wall
					g_xx[i] = g_xx[i] - g_xMax; g_vx[i] = g_vx[i]; g_vy[i] = g_vy[i];
				}
				if (g_yy[i] < 0.0) {
					g_yy[i] = 0.0; g_vx[i] = g_vx[i]; g_vy[i] = -g_vy[i];
				}
				if (g_yy[i] > g_yMax) {
					g_yy[i] = g_yMax; g_vx[i] = g_vx[i]; g_vy[i] = -g_vy[i];
				}
			}
		}

		/*
		if (g_periodicSW==1) {
			for (let i=0; i<nnp; i++) {
				if (g_kind[i]==1) {
					if (g_xx[i] < 0.0) {
						g_xx[i] = g_xx[i] + g_xMax;
					}
					if (g_xx[i] > g_xMax) {
						g_xx[i] = g_xx[i] - g_xMax;
					}
					if (g_yy[i] < 0.0) {
						g_yy[i] = g_yy[i] + g_yMax;
					}
					if (g_yy[i] > g_yMax) {
						g_yy[i] = g_yy[i] - g_yMax;
					}
				}
			}
		} else {
			for (let i=0; i<nnp; i++) {
				if (g_kind[i]==1) {
					if (g_xx[i] < 0.0) {
						g_xx[i] = 0.0; g_vx[i] = -g_vx[i]; g_vy[i] = g_vy[i];
					}
					if (g_xx[i] > g_xMax) {
						g_xx[i] = g_xMax; g_vx[i] = -g_vx[i]; g_vy[i] = g_vy[i];
					}
					if (g_yy[i] < 0.0) {
						g_yy[i] = 0.0; g_vx[i] = g_vx[i]; g_vy[i] = -g_vy[i];
					}
					if (g_yy[i] > g_yMax) {
						g_yy[i] = g_yMax; g_vx[i] = g_vx[i]; g_vy[i] = -g_vy[i];
					}
				}
			}
		}
		*/
	}

	function cellAttributeAt(x, y) {
		let ix = Math.floor(g_Nsx*x/g_xMax); if (ix>=g_Nsx) ix = g_Nsx-1;
		if (ix<0) ix = 0;
		let iy = Math.floor(g_Nsy*y/g_yMax); if (iy>=g_Nsy) iy = g_Nsy-1;
		if (iy<0) iy = 0;
		return g_cellAttribute[ix][iy];
	}

	function wallCell(i) {
		g_xx[i] -= g_vx[i]*g_dt;
		g_yy[i] -= g_vy[i]*g_dt;
		const at10 = cellAttributeAt(g_xx[i]+g_vx[i]*g_dt, g_yy[i]);
		const at01 = cellAttributeAt(g_xx[i], g_yy[i]+g_vy[i]*g_dt);
		if (at10==1) g_vx[i] = -g_vx[i];
		if (at01==1) g_vy[i] = -g_vy[i];
		if (at10==0 && at01==0) {
			g_vx[i] = -g_vx[i]; g_vy[i] = -g_vy[i];
		}
	}

	function absorbCell(i) {
		g_kind[i] = 0;
	}

	function adsorbCell(i) {
		g_kind[i] = -g_kind[i];
		g_xx[i] -= g_vx[i]*g_dt;
		g_yy[i] -= g_vy[i]*g_dt;
	}

	// --- set section

	function setSection() {
		const nsx=g_Nsx, nsy=g_Nsy, nnp=g_NNp;
		for (let i=0; i<nsx; i++) {
			for (let j=0; j<nsy; j++) {
				g_section[i][j][0] = 0;
			}
		}
		for (let ip=0; ip<nnp; ip++) {
			if (g_kind[ip]==1) {
				let i = Math.floor(nsx*g_xx[ip]/g_xMax); if (i>=nsx) i = nsx-1;
				let j = Math.floor(nsy*g_yy[ip]/g_yMax); if (j>=nsy) j = nsy-1;
				const iq = g_section[i][j][0]+1;
				g_section[i][j][0] = iq;
				g_section[i][j][iq] = ip;
			}
		}
	}

	function maxSection() {
		const nsx=g_Nsx,nsy=g_Nsy;
		let m=0;
		for (let i=0; i<nsx; i++) {
			for (let j=0; j<nsy; j++) {
				if (g_section[i][j][0]>m) m = g_section[i][j][0];
			}
		}
		return m;
	}

	// --- interaction

	function interaction() {
		const nsx=g_Nsx, nsy=g_Nsy, nnp=g_NNp;
		for (let i=0; i<nnp; i++) {
			g_mark[i] = 0;
		}
		const vmax = 2.0*maxSpeed();
		const kmmc = 0.5*g_ffn*g_sgm*vmax*g_dt/(g_dx*g_dy); // Mc = 0.5*Nc*Nc*FN*sgm*Vmax*dt/Lc^2
		for (let ic=0; ic<nsx; ic++) {
			for (let jc=0; jc<nsy; jc++) {
				const nn = g_section[ic][jc][0];
				if (nn>1) {
					const mmc = nn*nn*kmmc; // mmc: the number of collisions in the cell
					const immc = Math.floor(mmc);
					for (let ii=0; ii<immc; ii++) {
						collisionInTheCell(ic, jc, vmax);
					}
					if ( Math.random()<(mmc-immc) ) {
						collisionInTheCell(ic, jc, vmax);
					}
				}
			}
		}
	}

	function collisionInTheCell(ic, jc, vmax) {
		const nnc = g_section[ic][jc][0];
		let i, j;
		do {
			i = Math.floor(nnc*Math.random()); if (i>=nnc) i = nnc-1;
			j = Math.floor(nnc*Math.random()); if (j>=nnc) j = nnc-1;
		} while (i==j);
		const pi = g_section[ic][jc][i+1], pj = g_section[ic][jc][j+1];
		collision(pi,pj,vmax);
		g_mark[pi] = 1; g_mark[pj] = 1;
	}

	function collision(i, j, vmax) {
		const vrel = Math.sqrt((g_vx[i]-g_vx[j])*(g_vx[i]-g_vx[j])+(g_vy[i]-g_vy[j])*(g_vy[i]-g_vy[j]));
		if ( vrel/vmax > Math.random() ) {
			const a = g_mass[i]/(g_mass[i]+g_mass[j]), b = 1.0 - a;
			const vcmx = a*g_vx[i] + b*g_vx[j], vcmy = a*g_vy[i] + b*g_vy[j];
			const th = 2.0*g_PI*Math.random();
			const costh = Math.cos(th), sinth = Math.sin(th);
			g_vx[i] = vcmx + vrel*costh*b;
			g_vy[i] = vcmy + vrel*sinth*b;
			g_vx[j] = vcmx - vrel*costh*a;
			g_vy[j] = vcmy - vrel*sinth*a;
		}
	}

	// --- statictics

	function maxSpeed() {
		const nnp=g_NNp;
		let vm2 = 0.0;
		for (let i=0; i<nnp; i++) {
			if (g_kind[i]==1) {
				const v2 = g_vx[i]*g_vx[i]+g_vy[i]*g_vy[i];
				if (v2>vm2) vm2 = v2;
			}
		}
		return Math.sqrt(vm2);
	}

	function totalKineticEnergy() {
		const nnp=g_NNp;
		let tke=0.0;
		for (let i=0; i<nnp; i++) {
			if (g_kind[i]==1) {
				tke += 0.5*g_mass[i]*(g_vx[i]*g_vx[i]+g_vy[i]*g_vy[i]);
			}
		}
		return g_ffn*tke;
	}

	function numberOfPartickes(knd) {
		const nnp=g_NNp;
		let n=0;
		for (let i=0; i<nnp; i++) {
			if (g_kind[i]==knd) n += 1;
		}
		return n;
	}

	function numberOfCollided() {
		const nnp=g_NNp;
		let n=0;
		for (let i=0; i<nnp; i++) {
			if (g_mark[i]==1) n += 1;
		}
		return n;
	}

	function setVelocityDistribution() {
		const nnp=g_NNp;

		for (let i=0; i<200; i++) {
			g_vDistribution[i] = 0;
		}
		for (let i=0; i<nnp; i++) {
			if (g_kind[i]==1) {
				const v = Math.sqrt(g_vx[i]*g_vx[i]+g_vy[i]*g_vy[i]);
				if (v>1999.0) v = 1999.0;
				g_vDistribution[(int)(v/10.0)] += 1;
			}
		}
	}

	function setVSpace() {
		const nnp=g_NNp;

		for (let i=0; i<250; i++) {
			for (let j=0; j<250; j++) {
				g_vSpace[i][j] = 0;
			}
		}
		for (let ip=0; ip<nnp; ip++) {
			if (g_kind[ip]==1) {
				let i = (int)((g_vx[ip]+1250.0)/10.0+0.5);
				if (i<0) {
					i = 0;
				} else if (i>249) {
					i = 249;
				}
				let j = (int)((g_vy[ip]+1250.0)/10.0+0.5);
				if (j<0) {
					j = 0;
				} else if (j>249) {
					j = 249;
				}
				g_vSpace[i][j] += 1;
			}
		}
	}

	function setField() {
		const nsx=g_Nsx, nsy=g_Nsy;
		const kdens = g_ffn/(g_dx*g_dy);
		for (let i=0; i<nsx; i++) {
			for (let j=0; j<nsy; j++) {
				const n = g_section[i][j][0];
				g_meanN[i][j] = 0.9*g_meanN[i][j] + 0.1*n;
				g_density[i][j] = n*kdens;
				g_meanDensity[i][j] = 0.9*g_meanDensity[i][j] + 0.1*g_density[i][j];
				g_temperature[i][j] = 0.0;
				if (n>=1) {
					let vmx = 0.0, vmy = 0.0;
					for (let ip=1; ip<=g_section[i][j][0]; ip++) {
						const k = g_section[i][j][ip];
						vmx += g_vx[k]; vmy += g_vy[k];
					}
					vmx = vmx/n;
					vmy = vmy/n;
					g_xFlow[i][j] = vmx;
					g_yFlow[i][j] = vmy;
					g_meanxFlow[i][j] = 0.9*g_meanxFlow[i][j] + 0.1*vmx;
					g_meanyFlow[i][j] = 0.9*g_meanyFlow[i][j] + 0.1*vmy;
					let ke = 0.0;
					for (let ip=1; ip<=g_section[i][j][0]; ip++) {
						const k = g_section[i][j][ip];
						ke += 0.5*g_mass[k]*((g_vx[k]-vmx)*(g_vx[k]-vmx)+(g_vy[k]-vmy)*(g_vy[k]-vmy));
					}
					g_temperature[i][j] = ke/n/(1.0*g_kB);
				} else {
					g_xFlow[i][j] = 0.0;
					g_yFlow[i][j] = 0.0;
				}
				g_pressure[i][j] =  g_density[i][j]*g_kB*g_temperature[i][j];
			}
		}
	}

	function resetField() {
		const nsx=g_Nsx, nsy=g_Nsy;
		for (let i=0; i<nsx; i++) {
			for (let j=0; j<nsy; j++) {
				g_meanN[i][j] = 0.0;
				g_density[i][j] = 0.0;
				g_meanDensity[i][j] = 0.0;
				g_temperature[i][j] = 0.0;

				g_xFlow[i][j] = 0.0;
				g_yFlow[i][j] = 0.0;
				g_meanxFlow[i][j] = 0.0;
				g_meanyFlow[i][j] = 0.0;
				g_pressure[i][j] =  g_density[i][j]*g_kB*g_temperature[i][j];
			}
		}
	}


	// --------------------  public  --------------------

	return {
		init:			setInitialCondition,	// setInitialCondition( theme, boundary )
		evolve:			timeEvolution,			// timeEvolution( nCalc, setFieldSW )

		getSysParam:	function() { return [ g_NNp, g_Nsx, g_Nsy, g_dx, g_dt ]; },
		getNow:			function() { return [ g_sysTime, numberOfCollided() ]; },
		getPPosition:	function(i) { return [ g_xx[i], g_yy[i] ]; },
		getPVelocity:	function(i) { return [ g_vx[i], g_vy[i] ]; },
		getPKind:		function(i) { return g_kind[i]; },
		getCellKind:	function(i,j) { return g_cellAttribute[i][j]; }, // 0:free space 1:wall 2:absorption 3:adsorption
		getCellFlow:	function(i,j) { return [ g_xFlow[i][j], g_yFlow[i][j] ]; },
		getCellMeanFlow:function(i,j) { return [ g_meanxFlow[i][j], g_meanyFlow[i][j] ]; },
		getDensity:		function(i,j) { return g_density[i][j]; },
		getMeanDensity:	function(i,j) { return g_meanDensity[i][j]; },
		getTemperature:	function(i,j) { return g_temperature[i][j]; },
		getPressure:	function(i,j) { return g_pressure[i][j]; },
	};

})(); // ====================  jetDSMC2D end  ====================


const js088 = (function(){ // ====================  js Module  ====================

	const theModule = jetDSMC2D;
	const xCanvasSize = 500;	// in pixel
	const yCanvasSize = 480;	// in pixel
	let canvas;					// canvas2d
	let ctx;					// = canvas.getContext('2d');

	let v_theme = 0;			// no use
	let v_boundary = 1;			// fixed  ( 0: non periodic 1:periodic )
	let v_nCalc = 1;
	let v_setFieldSW = 1;

	let p_NNp, p_NNx, p_NNy, p_dx, p_dt; // <-- theModule.getSysParam();
	let sysTime, numberOfCollided;
	let nowData = [];
  let cellKindArray = [];
	let densArray = [];
	//let tempArray = [];
	//let pressArray = [];
  let vxArray = [];
	let vyArray = [];

	let xBoxSize = 420;
	let dispMode = 1; // 4: density
	let resetFlag = true;
	let pauseFlag = false;
	let stepFlag = false;
	//let inStepFlag = false;

  let breakFlag = false;
  let getFieldFlag = true;
  let fieldKind = 1 // cellKind, density, flow


	function main() {
		resetFlag = true;
		setCanvas( 'canvas_box', xCanvasSize, yCanvasSize );
		initDom();
		viewHome();

		animate();

		function setCanvas( canvasID, width, height ) {
			canvas = document.getElementById( canvasID );
			canvas.width  = width;
			canvas.height = height;
			ctx = canvas.getContext('2d');
			ctx.font = "16px 'sans-serif'";
			ctx.textBaseline = "bottom";
			ctx.textAlign = "left";
			ctx.lineWidth = 1;
			g3d.setMouseOnCanvas( canvas ); // 3D graphics
		}
	}


	function animate() {
    if ( breakFlag ) return;

		if ( resetFlag ) {
			resetFlag = false;
			theModule.init( v_theme, v_boundary ); // ( nn, BoxSizeInNM, contTemp )
			imgField = null;
			[ p_NNp, p_NNx, p_NNy, p_dx, p_dt ] = theModule.getSysParam();
			// g3d.init( NNx, NNy, NNz, dx, xCanvasSize, yCanvasSize, xBoxSize, yShift );
			g3d.init( p_NNx, p_NNy, p_NNy/2, p_dx, xCanvasSize, yCanvasSize, xBoxSize, 20 );
			g3d.drawWallGrid2D.threshold = 0.0;
			g3d.drawWallVectorField2D.threshold = 0.0;
      let getFieldFlag = true;
		}

		if ( !pauseFlag ) {
			theModule.evolve( v_nCalc, v_setFieldSW );
		} else if ( pauseFlag && stepFlag ) {
			stepFlag = false;
			theModule.evolve( v_nCalc, v_setFieldSW );
			//inStepFlag = true;
		}

		draw( ctx, dispMode );

    if ( getFieldFlag ) setFieldData( fieldKind );

		requestAnimationFrame(animate);
	}

  function setFieldData( fieldKind ) {
    if (fieldKind==1) { // fieldKind: 1: cellKind, density, flow
			cellKindArray = [];
			densArray = [];
			//tempArray = [];
			//pressArray = [];
			vxArray = [];
			vyArray = [];
			nowData = [ sysTime, numberOfCollided ];
      for (let i=0; i<p_NNx; i++) {
        cellKindArray[i] = [];
        densArray[i] = [];
        vxArray[i] = [];
				vyArray[i] = [];
        for (let j=0; j<p_NNy; j++) {
          cellKindArray[i][j] = theModule.getCellKind(i,j);
          densArray[i][j] = theModule.getMeanDensity(i,j);
					let vx, vy; [ vx, vy ] = theModule.getCellMeanFlow(i,j);
          vxArray[i][j] = vx;
					vyArray[i][j] = vy;
        }
      }
    }
  }


	// --------------------  draw  --------------------

	function draw( ctx, dispMode ) {

		const densMag = 2.5e-14, pressMag = 2.0e10, tempMag = 1.0e-4;
		const densityFunc = function(i,j) { return densMag*(theModule.getMeanDensity(i,j)) - g3d.cz0; };
		const wallFunc = function(i,j) { return (theModule.getCellKind(i,j)==1); };
		const pressFunc = function(i,j) { return pressMag*theModule.getPressure(i,j) - g3d.cz0; };
		const tempFunc = function(i,j) { return tempMag*theModule.getTemperature(i,j) - g3d.cz0; };

		const xp = 40, yp = 40, sc = 3; // imageData

		// clear
		ctx.clearRect(0, 0, xCanvasSize, yCanvasSize);

		if ( imgField == null ) {
			const xSize = p_NNx*g_imgScale, ySize = p_NNy*g_imgScale;
			imgField = ctx.getImageData( xp, yp, xSize, ySize );
		}

		if (dispMode==0) {
			dispText( "density(x,y)" );
			drawImgField( ctx, 0, 0, xp, yp, sc );
		} else if (dispMode==1) {
			dispText( "density(x,y) + flow" );
			drawImgField( ctx, 0, 1, xp, yp, sc );
		} else if (dispMode==2) { // vorticity + powder
			dispText( "pressure(x,y)" );
			drawImgField( ctx, 1, 0, xp, yp, sc );
		} else if (dispMode==3) { // speed + flow
			dispText( "temperature(x,y)" );
			drawImgField( ctx, 2, 0, xp, yp, sc );

		}  else if (dispMode==4) { // sample 500
			dispText( "DSMC particle sample N = 500" );
			drawWall(ctx, xp, yp, sc );
			const xBoxSize = 360;
			drawSample(ctx, 500, xp, yp, xBoxSize/(p_NNx*p_dx) );

		} else if (dispMode==5) {
			dispText( "grid: mean density(x,y)" );
			// g3d.drawWallGrid2D( ctx, rotAngle, wallFunc, zFunc, colorFactor, inc [, showBox] )
			g3d.drawWallGrid2D( ctx, 0.0, wallFunc, densityFunc, 0.5, 2 );
		} else if (dispMode==6) {
			dispText( "grid: temperature(x,y)" );
			// g3d.drawWallGrid2D( ctx, rotAngle, wallFunc, zFunc, colorFactor, inc [, showBox] )
			g3d.drawWallGrid2D( ctx, 0.0, wallFunc, tempFunc, 0.5, 2 );
		} else if (dispMode==7) {
			dispText( "vector grid: flow(x,y) + z:density" );
			const vFunc = function(i,j) {
				let vx, vy, mag=0.00001;
				[ vx, vy ] = theModule.getCellMeanFlow(i,j);
				return [ vx*mag, vy*mag ];
			}
			// g3d.drawWallVectorField2D(ctx, rotAngle, wallFunc, zFunc, vFunc, colorMode, inc [, showBox] )
			g3d.drawWallVectorField2D(ctx, 0.0, wallFunc, densityFunc, vFunc, 0, 2 );
		}

		[ sysTime, numberOfCollided ] = theModule.getNow();
		const xBox = p_NNx*p_dx, yBox = p_NNy*p_dx;
		ctx.fillStyle = "#888888";
		ctx.fillText(`box = ${xBox*100} x ${yBox*100} (cm),  time = ${(sysTime*1e6).toFixed(1)} (us)`,
						xp, yCanvasSize-30);
		ctx.fillText(`DSMC particles N = ${p_NNp},  collision = ${numberOfCollided} `, xp, yCanvasSize-10);
		//document.getElementById("text_caption").innerHTML = msg;

		function dispText( str ) {
			ctx.fillStyle = "#888888";
			ctx.fillText( str, xp, yCanvasSize-50 );
		}
	}


	// --- draw image field

	function drawImgField( ctx, FieldMode, addon, xp, yp, sc ) {
		const nnx=p_NNx, nny=p_NNy, densMag = 2.0e-10, pressMag = 5.0e10, tempMag = 1.0;

		ctx.strokeStyle = "rgb(80, 80, 80)"; // box
		ctx.strokeRect(xp-1,yp-1,p_NNx*sc+2,p_NNy*sc+2);

		for (let i=0; i<nnx; i++) {
			for (let j=0; j<nny; j++) {
				if (theModule.getCellKind(i,j)==1) continue;
				let hue;
				if (FieldMode==0) { // density
					hue = Math.floor(36300.0 - 1.0*densMag*theModule.getMeanDensity(i,j)) % 360;
				} else if (FieldMode==1) { // pressure
					hue = Math.floor(36300.0 - 1.0*pressMag*theModule.getPressure(i,j)) % 360;
				} else if (FieldMode==2) { // temperature
					hue = Math.floor(36300.0 - 1.0*tempMag*theModule.getTemperature(i,j)) % 360;
				}
				//ctx.fillStyle = `hsl(${hue},100%,50%)`;
				//ctx.fillRect(i*sc+xp,(nny-j-1)*sc+yp,sc,sc);
				setSquare(i*sc,(nny-j-1)*sc,sc,hue);
			}
		}
		ctx.putImageData( imgField, xp, yp );
		if (addon==1) drawFlow( ctx, 0.05, xp, yp, sc );
		if (addon==2) drawPowder( ctx, p_nPowder, xp, yp, sc );
	}

	function drawFlow( ctx, mag, xp, yp, sc ) {
		const nnx=p_NNx, nny=p_NNy;

		ctx.lineWidth = 1;
		for (let i=0; i<nnx; i+=2) {
			for (let j=0; j<nny; j+=2) {
				if ( theModule.getCellKind(i,j)==0 ) { // 0:free space, 1:wall
					let vx, vy;
					[ vx, vy ] = theModule.getCellFlow(i,j);
					const color = (vx>=0) ? "#6666ff" : "#ff4488";
					const x = (i+0.5)*sc, y = (nny-j-0.5)*sc;
					g3d.drawLine( ctx, xp+x, yp+y, xp+x+vx*mag, yp+y-vy*mag, color );
				}
			}
		}
	}

	function drawPowder(ctx, nPowder, xp, yp, sc ) {
		const nny=p_NNy;

		ctx.fillStyle = "#ff80ff";
		for (let i=0; i<nPowder; i++) {
			let x, y;
			[ x, y ] = theModule.getPowderPos(i);
			ctx.fillRect( xp+x*sc, yp+(nny-y)*sc, 2, 2 );
		}
	}


	// --- draw DSMC particle samples

	function drawSample(ctx, nn, xp, yp, scale ) {
		const tau=100*p_dt, yMax=p_NNy*p_dx;

		for (let i=0; i<nn; i++) {
			let x, y;
			[ x, y ] = theModule.getPPosition(i);
			g3d.drawDisc( ctx, x*scale+xp, (yMax-y)*scale+yp, 2, "#00ff00" );
		}
		for (let i=0; i<nn; i++) {
			let x, y, vx, vy;
			[ x, y ] = theModule.getPPosition(i);
			[ vx, vy ] = theModule.getPVelocity(i);
			const colr = (vx>=0) ? "#2222ff" : "#ff2222";
			g3d.drawLine(ctx, x*scale+xp, (yMax-y)*scale+yp, (x+vx*tau)*scale+xp, (yMax -y-vy*tau)*scale+yp, colr);
		}
	}

	function drawWall(ctx, xp, yp, sc ) {
		const nnx=p_NNx, nny=p_NNy;
		ctx.fillStyle = "#666666";
		ctx.fillRect(xp,yp,p_NNx*sc,p_NNy*sc);
		ctx.fillStyle = "#333333";
		for (let i=0; i<nnx; i++) {
			for (let j=0; j<nny; j++) {
				if (theModule.getCellKind(i,j)==1) {
					ctx.fillRect(i*sc+xp,(nny-j-1)*sc+yp,sc,sc);
				}
			}
		}
	}


	// -------------------- image field modue  --------------------
	// ver 0.0.0  2023.04.20

	let imgField = null;		// image in draw()
	const g_imgScale = 3;
	const g_hue = dimInt2(3,361); // red:hue[0][deg]  green:hue[1][deg]  blue:hue[2][deg]
	const g_hueColor = [];
	setHueColor();

	function dimInt2(ni,nj) {
		let a=[];
		for (let i=0; i<ni; i++) {
			a[i] = [];
			for (let j=0; j<nj; j++) {
				a[i][j] = 0;
			}
		}
		return a;
	}

	function setHueColor() {
		if (setHueColor.ok==true) return;
		let x,r,g,b;
		for (let deg=0; deg<361; deg++) {
			if (deg<60) {
				x = deg;
				g_hue[0][deg] = 255;
				g_hue[1][deg] = Math.floor(255.0*x/60.0);
				g_hue[2][deg] = 0;
			} else if (deg<120) {
				x = deg-60;
				g_hue[0][deg] = Math.floor(255.0*(60.0-x)/60.0);
				g_hue[1][deg] = 255;
				g_hue[2][deg] = 0;
			} else if (deg<180) {
				x = deg-120;
				g_hue[0][deg] = 0;
				g_hue[1][deg] = 255;
				g_hue[2][deg] = Math.floor(255.0*x/60.0);
			} else if (deg<240) {
				x = deg-180;
				g_hue[0][deg] = 0;
				g_hue[1][deg] = Math.floor(255.0*(60.0-x)/60.0);
				g_hue[2][deg] = 255;
			} else if (deg<360) {
				x = deg-240;
				g_hue[0][deg] = Math.floor(255.0*x/120.0);
				g_hue[1][deg] = 0;
				g_hue[2][deg] = Math.floor(255.0*(120.0-x)/120.0);
			} else {
				g_hue[0][deg] = 0; g_hue[1][deg] = 0; g_hue[2][deg] = 0;
			}
			g_hueColor[deg] = 'rgb('+g_hue[0][deg]+','+g_hue[1][deg]+','+g_hue[2][deg]+')';
		}
		setHueColor.ok = true;
	}
	setHueColor.ok = false;

	function setSquare(ix,iy,aSize,colIndex) {
		const xSize = p_NNx*g_imgScale, data = imgField.data;

		for (let j=iy; j<iy+aSize; j++) {
			for (let i=ix; i<ix+aSize; i++) {
				const idx = (j*xSize+i)*4;
				data[idx] = g_hue[0][colIndex];
				data[idx+1] = g_hue[1][colIndex];
				data[idx+2] = g_hue[2][colIndex];
				data[idx+3] = 255;
			}
		}
	}


	// --------------------  graphics 3D (field) module  --------------------
	//
	// ver 0.0.1  2018.12.16  last updated on 2023.03.01
	// ver 0.0.2  2023.03.03  last updated on 2023.06.01

	let g_NNx, g_NNy, g_NNz, g_dx, g_dy, g_dz, g_xCanvasSize, g_yCanvasSize, g_xBoxSize, g_yShift;

	const g3d = {};				// namespace of graphic 3D module

	g3d.mouseDownFlag = 0;		// 1:on mouse down, 0:else
	g3d.x_mouse = 0;			// x-position of mouse
	g3d.y_mouse = 0;			// y-position of mouse
	g3d.x0_mouse = 0;			// drag-started x-position of mouse
	g3d.y0_mouse = 0;			// drag-started y-position of mouse
	g3d.zoom = 1.0;

	g3d.xMax = 0.0;				// x-length of box
	g3d.yMax = 0.0;				// y-length of box
	g3d.zMax = 0.0;				// z-length of box
	g3d.cx0 = 0.0;				// x-component of rotate center
	g3d.cy0 = 0.0;				// y-component of rotate center
	g3d.cz0 = 0.0;				// z-component of rotate center
	g3d.Ax = -Math.PI/15.0;		// rotate angle around x-axis
	g3d.Ay = -Math.PI/15.0;		// rotate angle around y-axis
	g3d.ddAy = 0.0;				// Ay change rate for auto-rotate: eg. dday=0.5*Math.PI/180
	g3d.cosAx = 0.0;			// cosAx=Math.cos(Ax)
	g3d.sinAx = 0.0;			// sinAx=Math.sin(Ax)
	g3d.cosAy = 0.0;			// cosAy=Math.cos(Ay)
	g3d.sinAy = 0.0;			// sinAy=Math.sin(Ay)

	g3d.xApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.yApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.zApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.pxApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.pyApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.pzApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.boxApex = [[0,0,0], [1,0,0], [0,1,0], [1,1,0], [0,0,1], [1,0,1], [0,1,1], [1,1,1] ];
	g3d.boxEdge = [[0,1,9], [0,2,9], [0,4,9], [1,3,9], [1,5,9], [2,3,9],
					[2,6,9], [3,7,9], [4,5,9], [4,6,9], [5,7,9], [6,7,9] ];

	//--- set mouse on canvas

	// g3d.setMouseOnCanvas( canvas );
	g3d.setMouseOnCanvas = function( canvas ) {
		canvas.addEventListener('mousemove', g3d.mouse_move);
		canvas.addEventListener('mousedown', g3d.mouse_down);
		canvas.addEventListener('mouseup', g3d.mouse_up);
		//canvas.addEventListener("mousewheel", g3d.mouseWheel);
	};

	g3d.mouse_move = function(e) {
		const pi = Math.PI;

		if (g3d.mouseDownFlag==1) {
			g3d.x_mouse = e.clientX;
			g3d.y_mouse = e.clientY;
			g3d.Ay = g3d.Ay + 0.5*(g3d.x_mouse-g3d.x0_mouse)*pi/180;
			if (g3d.Ay<-pi) g3d.Ay += 2*pi;
			if (g3d.Ay>pi) g3d.Ay += -2*pi;
			//g3d.Ax = g3d.Ax + 0.5*(g3d.y_mouse-g3d.y0_mouse)*pi/180;
			g3d.Ax = g3d.Ax - 0.5*(g3d.y_mouse-g3d.y0_mouse)*pi/180;
			if (g3d.Ax<-0.5*pi) g3d.Ax = -0.5*pi;
			if (g3d.Ax>0.5*pi) g3d.Ax = 0.5*pi;
			g3d.x0_mouse = g3d.x_mouse;
			g3d.y0_mouse = g3d.y_mouse;
		}
	};

	g3d.mouse_down = function(e) {
		if (g3d.mouseDownFlag==0) {
			g3d.x0_mouse = e.clientX;
			g3d.y0_mouse = e.clientY;
			g3d.x_mouse = g3d.x0_mouse;
			g3d.y_mouse = g3d.y0_mouse;
			g3d.mouseDownFlag = 1;
		}
	};

	g3d.mouse_up = function(e) {
		if (g3d.mouseDownFlag==1) {
			g3d.mouseDownFlag = 0;
		}
	};

	g3d.mouseWheel = function(e) {
		g3d.deltaY = e.deltaY;
		if ( g3d.deltaY > 0 ) g3d.zoom *= 0.95;
		else if ( g3d.deltaY < 0 ) g3d.zoom *= 1.05;
		if ( g3d.zoom<0.5 ) g3d.zoom = 0.5;
		if ( g3d.zoom>2.0 ) g3d.zoom = 2.0;
	};

	//--- 3D graphics aid

	// g3d.init( NNx, NNy, NNz, dx, xCanvasSize, yCanvasSize, xBoxSize, yShift );
	g3d.init = function( NNx, NNy, NNz, dx, xCanvasSize, yCanvasSize, xBoxSize, yShift ) {
		g_NNx = NNx; g_NNy = NNy; g_NNz = NNz;
		g_dx = dx; g_dy = dx; g_dz = dx;
		g_xCanvasSize = xCanvasSize; g_yCanvasSize = yCanvasSize;
		g_xBoxSize = (xBoxSize==undefined) ? 300 : xBoxSize;
		g_yShift = (yShift==undefined) ? 20 : yShift;
		g3d.setSize();
	}

	g3d.setSize = function() {
		g3d.xMax = g_NNx*g_dx;		// x-length of box
		g3d.yMax = g_NNy*g_dy;		// y-length of box
		g3d.zMax = g_NNz*g_dz;		// z-length of box
		g3d.cx0 = 0.5*g3d.xMax;		// x-component of rotate center
		g3d.cy0 = 0.5*g3d.yMax;		// y-component of rotate center
		g3d.cz0 = 0.5*g3d.zMax;		// z-component of rotate center
	};

	// g3d.setRotateAngle( AxInDegree, AyInDegree );
	g3d.setRotateAngle = function( AxInDegree, AyInDegree ) {
		g3d.Ax = AxInDegree*Math.PI/180.0;
		g3d.Ay = AyInDegree*Math.PI/180.0;
	};

	// g3d.scxpypFunc();
	g3d.scxpypFunc = function() {
		const xBoxSize = g_xBoxSize;
		const xCenter = g_xCanvasSize/2, yCenter = g_yCanvasSize/2-g_yShift, yBoxSize = (xBoxSize/g_NNx)*g_NNy;
		const xp = xCenter - (xBoxSize/2)*g3d.zoom, yp = yCenter - (yBoxSize/2)*g3d.zoom; // g3d param
		const sc = xBoxSize/(g_NNx*g_dx)*g3d.zoom;
		return [ sc, xp, yp ];
	}

	g3d.set3DRotateXY = function(rotateRateOfAyInDegree) {
		g3d.ddAy = rotateRateOfAyInDegree*Math.PI/180.0;
		g3d.Ay= g3d.Ay + g3d.ddAy; // auto-rotate : if (ddAy==0.0), stop
		if (g3d.Ay>Math.PI) g3d.Ay = g3d.Ay - 2.0*Math.PI;
		if (g3d.Ay<-Math.PI) g3d.Ay = g3d.Ay + 2.0*Math.PI;
		g3d.setBox();           // set box apex
		g3d.setRotateXY(g3d.Ax,g3d.Ay); // set rotate param
		g3d.rotateApexXY();     // box Apex rotate--> pxApex[i],pyApex[i],pzApex[i]
		g3d.markFarEdge();      // boxEdge[iEdge][2]=1:far side edge or 0:near side edge
	};

	g3d.setBox = function() {
		for (let i=0; i<8; i++) {
			g3d.xApex[i] = g3d.boxApex[i][0]*g3d.xMax;
			g3d.yApex[i] = g3d.boxApex[i][1]*g3d.yMax;
			g3d.zApex[i] = g3d.boxApex[i][2]*g3d.zMax;
		}
	};

	g3d.setRotateXY = function(angleX,angleY) {
		g3d.cosAx = Math.cos(angleX);
		g3d.sinAx = Math.sin(angleX);
		g3d.cosAy = Math.cos(angleY);
		g3d.sinAy = Math.sin(angleY);
		g3d.cx0 = 0.5*g3d.xMax;
		g3d.cy0 = 0.5*g3d.yMax;
		g3d.cz0 = 0.5*g3d.zMax;
	};

	g3d.rotateApexXY = function() { // rotate box apex
		const cosAx=g3d.cosAx,sinAx=g3d.sinAx,cosAy=g3d.cosAy,sinAy=g3d.sinAy,cx0=g3d.cx0,cy0=g3d.cy0,cz0=g3d.cz0;

		for (let i=0; i<8; i++) {
			g3d.pxApex[i] = cosAy*(g3d.xApex[i]-cx0)+sinAy*(sinAx*(g3d.yApex[i]-cy0)+cosAx*(g3d.zApex[i]-cz0))+cx0;
			g3d.pyApex[i] = cosAx*(g3d.yApex[i]-cy0)-sinAx*(g3d.zApex[i]-cz0) + cy0;
			g3d.pzApex[i] =-sinAy*(g3d.xApex[i]-cx0)+cosAy*(sinAx*(g3d.yApex[i]-cy0)+cosAx*(g3d.zApex[i]-cz0))+cz0;
		}
	};

	g3d.markFarEdge = function() {
		//seek far apex --> iMin
		let zMin = g3d.pzApex[0];
		let iMin = 0;
		for (let i=1; i<8; i++) {
			if (zMin>g3d.pzApex[i]) {
				zMin = g3d.pzApex[i];
				iMin = i;
			}
		}
		//mark far edge
		for (let iEdge=0; iEdge<12; iEdge++) {
			g3d.boxEdge[iEdge][2] = 0;
			if (g3d.boxEdge[iEdge][0]==iMin || g3d.boxEdge[iEdge][1]==iMin) g3d.boxEdge[iEdge][2] = 1;
		}
	};

	g3d.drawRotatedDisc = function(ctx, x,y,z,r,color,sc,xp,yp) {
		const cosAx=g3d.cosAx,sinAx=g3d.sinAx,cosAy=g3d.cosAy,sinAy=g3d.sinAy,cx0=g3d.cx0,cy0=g3d.cy0,cz0=g3d.cz0;

		const x1 = cosAy*(x-cx0)+sinAy*(sinAx*(y-cy0)+cosAx*(z-cz0)) + cx0
		const y1 = cosAx*(y-cy0)-sinAx*(z-cz0) + cy0
		//z1 =-sinAy*(x-cx0)+cosAy*(sinAx*(y-cy0)+cosAx*(z-cz0)) + cz0
		g3d.drawDisc(ctx, x1*sc+xp,y1*sc+yp,r,color);
	};

	g3d.drawRotatedLine = function(ctx, x1,y1,z1,x2,y2,z2,color,sc,xp,yp) {
		const cosAx=g3d.cosAx,sinAx=g3d.sinAx,cosAy=g3d.cosAy,sinAy=g3d.sinAy,cx0=g3d.cx0,cy0=g3d.cy0,cz0=g3d.cz0;

		const x1p = cosAy*(x1-cx0)+sinAy*(sinAx*(y1-cy0)+cosAx*(z1-cz0)) + cx0
		const y1p = cosAx*(y1-cy0)-sinAx*(z1-cz0) + cy0
		const x2p = cosAy*(x2-cx0)+sinAy*(sinAx*(y2-cy0)+cosAx*(z2-cz0)) + cx0
		const y2p = cosAx*(y2-cy0)-sinAx*(z2-cz0) + cy0
		g3d.drawLine(ctx, x1p*sc+xp,y1p*sc+yp,x2p*sc+xp,y2p*sc+yp,color);
	};

	g3d.plotNearEdge = function(ctx, sc,xp,yp,color) {
		for (let iEdge=0; iEdge<12; iEdge++) {
			if (g3d.boxEdge[iEdge][2]==0) { //far edge mark = 1
				g3d.plotEdge(ctx, iEdge,sc,xp,yp,color);
			}
		}
	};

	g3d.plotFarEdge = function(ctx, sc,xp,yp,color) {
		for (let iEdge=0; iEdge<12; iEdge++) {
			if (g3d.boxEdge[iEdge][2]==1) { //far edge mark = 1
				g3d.plotEdge(ctx, iEdge,sc,xp,yp,color);
			}
		}
	};

	g3d.plotEdge = function(ctx, iEdge,sc,xp,yp,color) {
		let iApex = g3d.boxEdge[iEdge][0];
		const x1=g3d.pxApex[iApex]*sc+xp, y1=g3d.pyApex[iApex]*sc+yp;
		iApex = g3d.boxEdge[iEdge][1];
		const x2=g3d.pxApex[iApex]*sc+xp, y2=g3d.pyApex[iApex]*sc+yp;
		g3d.drawLine(ctx, x1, y1, x2, y2, color);
	};

	g3d.drawLine = function(ctx, x1, y1, x2, y2, color) {
		ctx.strokeStyle = color;
		ctx.beginPath();
		ctx.moveTo(x1, y1);
		ctx.lineTo(x2, y2);
		ctx.stroke();
	};

	g3d.drawDisc = function(ctx, x, y, r, color) {
		ctx.fillStyle = color;
		ctx.beginPath();
		ctx.arc(x, y, r, 0, 2*Math.PI, false);
		ctx.fill();
	};

	// --------------------  end of graphics 3D (field) module  --------------------

	// g3d_extension grid2d  created 2023.06.17, last updated 2023.06.17
	// g3d.drawWallGrid2D( ctx, rotAngle, wallFunc, zFunc, colorFactor, inc [, showBox] )
	g3d.drawWallGrid2D = function ( ctx, rotAngle, wallFunc, zFunc, colorFactor, inc, showBox ) {
		let sc, xp, yp; [ sc, xp, yp ] = g3d.scxpypFunc();
		const nnx = g_NNx, nny = g_NNy, threshold = g3d.drawWallGrid2D.threshold;

		g3d.set3DRotateXY(rotAngle);
		if ( (showBox & 1)>0 || showBox==undefined ) g3d.plotFarEdge(ctx, sc,xp,yp,"#444400"); // dark yellow
		for (let jj=0; jj<nny; jj+=inc) {
			let j=jj;if (g3d.pzApex[2]-g3d.pzApex[0]<0) j=nny-jj-1;
			for (let ii=0; ii<nnx; ii++) {
				let i=ii;if (g3d.pzApex[1]-g3d.pzApex[0]<0) i=nnx-ii-1;
				if (i<0 || i+1>nnx-1) continue;

				if ( wallFunc(i,nny-j-1) ) {
					const  x = i*g_dx, y = j*g_dx, z = 0, z1 = 2*g3d.cz0;
					g3d.drawRotatedLine(ctx, x,y,z,x,y,z1,'#888888',sc,xp,yp);
					continue;
				}

				const f = zFunc(i,nny-j-1);
				const x = i*g_dx, y = j*g_dy, z = f + g3d.cz0;
				const f1 = zFunc(i+1,nny-j-1);
				const x1 = (i+1)*g_dx, y1 = j*g_dy, z1 = f1 + g3d.cz0;
				let colr;
				if ( typeof(colorFactor)=='number' ) {
					const th = (18120 - Math.floor(colorFactor*180.0*(f+f1)/g3d.cz0))%360;
					const a = Math.abs((f+f1)/g3d.cz0);
					colr = (a>threshold) ? `hsl(${th},100%,50%)` : "#444444" ;
				} else if ( typeof(colorFactor)=='string' ) {
					colr = colorFactor;
				} else if ( typeof(colorFactor)=='function' ) {
					colr = colorFactor(i,nny-j-1);
				}
				g3d.drawRotatedLine(ctx, x,y,z,x1,y1,z1,colr,sc,xp,yp);
			}
		}
		for (let ii=0; ii<nnx; ii+=inc) {
			let i=ii;if (g3d.pzApex[1]-g3d.pzApex[0]<0) i=nnx-ii-1;
			for (let jj=0; jj<nny; jj++) {
				let j=jj;if (g3d.pzApex[2]-g3d.pzApex[0]<0) j=nny-jj-1;
				if (j<0 || j+1>nny-1) continue;

				if ( wallFunc(i,nny-j-1) ) {
					continue;
				}

				const f = zFunc(i,nny-j-1);
				const x = i*g_dx, y = j*g_dy, z = f + g3d.cz0;
				const f1 = zFunc(i,nny-j-2);
				const x1 = i*g_dx, y1 = (j+1)*g_dy, z1 = f1 + g3d.cz0;
				let colr;
				if ( typeof(colorFactor)=='number' ) {
					const th = (18120 - Math.floor(colorFactor*180.0*(f+f1)/g3d.cz0))%360;
					const a = Math.abs((f+f1)/g3d.cz0);
					colr = (a>threshold) ? `hsl(${th},100%,50%)` : "#444444" ;
				} else if ( typeof(colorFactor)=='string' ) {
					colr = colorFactor;
				} else if ( typeof(colorFactor)=='function' ) {
					colr = colorFactor(i,nny-j-1);
				}
				g3d.drawRotatedLine(ctx, x,y,z,x1,y1,z1,colr,sc,xp,yp);
			}
		}
		if ( (showBox & 2)>0 || showBox==undefined ) g3d.plotNearEdge(ctx, sc,xp,yp,"#999900"); //yellow

		ctx.font = "12px 'sans-serif'";
		ctx.fillStyle = "#888888";
		ctx.fillText(`Ax=${(g3d.Ax*180/Math.PI).toFixed(1)}, Ay=${(g3d.Ay*180/Math.PI).toFixed(1)}`, 10, 15);
		ctx.font = "16px 'sans-serif'";
	};
	g3d.drawWallGrid2D.threshold = 0.005;

	// g3d_extension gvextor field  created 2023.06.17, last updated 2023.06.17
	// g3d.drawWallVectorField2D(ctx, rotAngle, wallFunc, zFunc, vFunc, colorMode, inc [, showBox] )
	g3d.drawWallVectorField2D = function(ctx, rotAngle, wallFunc, zFunc, vFunc, colorMode, inc, showBox ) {
		let sc, xp, yp; [ sc, xp, yp ] = g3d.scxpypFunc();
		const nnx=g_NNx, nny=g_NNy, nnz=g_NNz, dx=g_dx, threshold = g3d.drawWallVectorField2D.threshold;

		g3d.set3DRotateXY(rotAngle);
		if ( (showBox & 1)>0 || showBox==undefined ) g3d.plotFarEdge(ctx, sc,xp,yp,"#444400"); // dark yellow
		for (let ii=0; ii<nnx; ii+=inc) {
			let i=ii; if (g3d.pzApex[1]-g3d.pzApex[0]<0) i=nnx-ii-1;
			for (let jj=0; jj<nny; jj+=inc) {
				let j=jj; if (g3d.pzApex[2]-g3d.pzApex[0]<0) j=nny-jj-1;

				if ( wallFunc(i,nny-j-1) ) {
					const  x = i*dx, y = j*dx, z = 0, z1 = 2*g3d.cz0;
					g3d.drawRotatedLine(ctx, x,y,z,x,y,z1,'#888888',sc,xp,yp);
					continue;
				}

				const f = zFunc(i,nny-j-1);
				const x = i*dx, y = j*dx, z = f + g3d.cz0;
				let vx, vy;
				[ vx, vy ] = vFunc(i,nny-j-1);
				if ( vx*vx+vy*vy < threshold ) continue;
				const x1 = x + vx, y1 = y -vy, z1 = z;
				let colr;
				if ( colorMode==0 ) {
					const hue = Math.floor((Math.atan2(vy,vx))*180.0/Math.PI+180.0);
					colr = `hsl(${hue},100%,50%)`;
				} else if ( colorMode==1 ) {
					colr = (vx>0) ? "#3333ff" : "#ff3333";
				}
				g3d.drawRotatedLine(ctx, x,y,z,x1,y1,z1,colr,sc,xp,yp);
			}
		}
		if ( (showBox & 2)>0 || showBox==undefined ) g3d.plotNearEdge(ctx, sc,xp,yp,"#999900"); //yellow

		ctx.font = "12px 'sans-serif'";
		ctx.fillStyle = "#888888";
		ctx.fillText(`Ax=${(g3d.Ax*180/Math.PI).toFixed(1)}, Ay=${(g3d.Ay*180/Math.PI).toFixed(1)}`, 10, 15);
		ctx.font = "16px 'sans-serif'";
	};
	g3d.drawWallVectorField2D.threshold = 0.5;


	// --------------------  control  --------------------

	function initDom() {
		document.getElementById("step_button").style.visibility = "hidden";
	}

	function reset() { resetFlag = true; }

	function pause() {
		let btn = document.getElementById("pause_button");

		pauseFlag = ( pauseFlag==false );
		if ( pauseFlag==false ) btn.innerHTML = "pause"; else btn.innerHTML = "go";

		if ( pauseFlag==true ) {
			document.getElementById("step_button").style.visibility = "visible";
		} else {
			document.getElementById("step_button").style.visibility = "hidden";
		}
	}

	function step() { stepFlag = true; }

	function setTheme() {  // select theme
		v_theme = 0 + document.getElementById("slct_theme").selectedIndex;
		resetFlag = true;
	}

	function setRevL() {
		const sr = 0 + document.getElementById("slct_RevL").selectedIndex;
		v_RevL = 50.0*Math.pow(2,sr);
		resetFlag = true;
	}

	function setDispMode() {  // select dispMode
		dispMode = 0 + document.getElementById("slct_dispMode").selectedIndex;
	}

	function setSpeed() {
		v_nCalc = 1 + document.getElementById("slct_speed").selectedIndex;
	}

	function viewHome() {
		g3d.setRotateAngle(30,-15);
		g3d.zoom = 1.0;
	}

  // function controlled by python

  function breakLoop() {
    breakFlag = true;
  }

  function pysetTheme( theme ) {
    v_theme = theme
    document.getElementById("slct_theme").selectedIndex = theme;
    resetFlag = true;
  }

  function pysetDispMode( mode ) {
    dispMode = mode;
    document.getElementById("slct_dispMode").selectedIndex = mode;
  }

  function pygetData( pyMsg ) {
    document.getElementById("text_from_python").innerHTML = pyMsg;
    return [ sysTime, numberOfCollided ];
  }

  function pygetFieldData() {
    getFieldFlag = false;
    return [ nowData, cellKindArray, densArray, vxArray, vyArray ];
  }


	// --------------------  public  --------------------

	return {
		main:			main,			// main()

		reset:			reset,			// reset()
		pause:			pause,			// pause()
		step:			step,			// step()

		setTheme:		setTheme,		// setTheme()
		setRevL:		setRevL,		// setRevL()
		setDispMode:	setDispMode,	// setDispMode()
		setSpeed:		setSpeed,		// setSpeed()
		viewHome:		viewHome,		// viewHome()

    breakLoop: breakLoop, // breakLoop();
    pysetTheme: pysetTheme, // pysetTheme( theme )
    pysetDispMode: pysetDispMode, // pysetDispMode( mode )
    pygetData: pygetData, // pygetData( pyMsg ) : return [ sysTime, numberOfCollided ]
    pygetFieldData: pygetFieldData, // pygetFieldData() : return [ nowData, cellKindArray, densArray, vxArray, vyArray ]
	};

})(); // ====================  js088 module end  ====================


const js = js088;
//window.addEventListener('load', js.main );
js.main();


// %%%%%%%%%%%%%%%%%%%%  end of javaScript  %%%%%%%%%%%%%%%%%%%%

</script>

<style type="text/css">
    body { text-align:left; color:#000000; background-color:#fff8dd; }
</style>

</head>

<body>
<p>[js088] rarefied gas jet - direct simulation Monte Carlo DSMC2D</p>
<canvas ID="canvas_box" style="background-color: #000000;" WIDTH="500" HEIGHT="480"></canvas>
<br>

<label>theme: rarefied gas jet</label>
    <span style="margin-right: 160px;"></span>
<button onClick="js.reset()">reset</button>
    <span style="margin-right: 20px;"></span>
<button id="pause_button" onClick="js.pause()">pause</button>
    <span style="margin-right: 10px;"></span>
<button id="step_button" onClick="js.step()">step</button>
<br>

<label>disp mode:</label>
<select id="slct_dispMode" onChange="js.setDispMode()">
<option>density(x,y)</option><option selected>density(x,y) + flow</option>
<option>pressure(x,y)</option><option>temperature(x,y)</option>
<option>dsmc particle sample 500</option>
<option>grid mean density(x,y)</option><option>grid temperature(x,y)</option>
<option>flow(x,y) z:density(x,y)</option>
</select>
    <span style="margin-right: 20px;"></span>
<label>nCalc/frame:</label>
<select id="slct_speed" onChange="js.setSpeed()">
<option selected>1</option><option>2</option><option>3</option><option>4</option>
</select>
<br>

<button onClick="js.viewHome()">return to initial view</button>
<br>

<p id="text_caption" ></p>
<hr width="480" align="left" color="#a0a0a0">
<button onClick="js.breakLoop()">animation break to END</button>
    <span style="margin-right: 50px;"></span> python msg:
<span id="text_from_python" ></span>
<br>


</body>
</html>


  ''')
  display(htm)
# end def


In [ ]:
# exec html-js code
exec_html_js()
print("--- push [animation break to END] button to end ---")

In [ ]:
# get data and print

import time

# exec html-js code
exec_html_js()
print("-- start --")

# get data and print
for i in range(10):
  [ sysTime, numberOfCollided ] = eval_js( 'js.pygetData({})'.format(i) )
  print(f'i = {i:>2d},  time = {sysTime*1e6:>6.1f} (us),  DSMC particles total ccollided: {numberOfCollided:>8d}')
  time.sleep(3)

time.sleep(1)
# animation break to END
eval_js( 'js.breakLoop()' )
print("-- end --")

In [ ]:
# change dispMode

import time

dispModeList = [
  '0: density(x,y)', '1: density(x,y) + flow', '2: pressure(x,y)', '3: temperature(x,y)', '4: dsmc particle sample 500',
  '5: grid mean density(x,y)', '6: grid temperature(x,y)', '7: flow(x,y) z:density(x,y)' ]

# exec html-js code
exec_html_js()
print("-- start --")

# change dispMode and print
for dispMode in [ 0, 1, 2, 3, 4, 5, 6, 7 ]:
    eval_js( 'js.pysetDispMode({})'.format(dispMode) )
    print("-- dispMode:", dispModeList[dispMode], " --")
    [ sysTime, numberOfCollided ] = eval_js( 'js.pygetData({})'.format(dispMode) )
    print( f'\t time = {sysTime*1e6:>6.1f} (us),  DSMC particles total ccollided: {numberOfCollided:>8d}' )
    time.sleep(3)

time.sleep(1)
# animation break to END
eval_js( 'js.breakLoop()' )
print("-- end --")

In [ ]:
# exec html-js code, and python control / get field and vector array and plot

import time
import numpy as np

# exec html-js code
exec_html_js()
print("-- start --")

# set theme
# change dispMode
dispMode = 7 # '7: flow(x,y) z:density(x,y)'
eval_js( 'js.pysetDispMode({})'.format(dispMode) )
print("-- set dispMode:", dispModeList[dispMode], " --")

# get data and print
for i in range(10):
  [ sysTime, numberOfCollided ] = eval_js( 'js.pygetData({})'.format(i) )
  print( f'i = {i:>2d},  time = {sysTime*1e6:>6.1f} (us),  DSMC particles total ccollided: {numberOfCollided:>8d}' )
  time.sleep(3)

# get field data
print("-- get pressure data array --")
[ nowData, cellKindArray, densArray, vxArray, vyArray ] = eval_js('js.pygetFieldData()')
[ sysTime, numberOfCollided ] = nowData
print( f'\t got time = {sysTime*1e6:>6.1f} (us),  DSMC particles total ccollided: {numberOfCollided:>8d}' )

Kind = np.array(cellKindArray).astype(int)
Dens = np.array(densArray)
U = np.array(vxArray)
V = np.array(vyArray)
print(f'shape Kind:{Kind.shape}, Dens:{Dens.shape}, U:{U.shape}, V:{V.shape} ')

# animation break to END
eval_js('js.breakLoop()')
print("-- end --")


In [ ]:
# save field data

import numpy as np

# convert [ cellKindArray, densArray, vxArray, vyArray ] to np_data
np_data = np.array([ cellKindArray, densArray, vxArray, vyArray ])
print("shape of  np_data :", np_data.shape )

# save np_data
print("-- save as 'js088_data.npy'" )
np.save( 'js088_data.npy', np_data )

In [ ]:
# density - image plot

import numpy as np
import matplotlib.pyplot as plt

Z = Dens.T
im = plt.imshow(Z, origin='lower', cmap='jet' )
plt.colorbar(im)
plt.title("density map")
plt.show()

In [ ]:
# density - surface plot

import numpy as np
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(projection='3d')

# Plot the surface
nx, ny = Dens.shape
x = np.arange(0,nx)
y = np.arange(0,ny)
X, Y = np.meshgrid(x, y)
Z = Dens.T
ax.plot_surface(X,Y,Z, cmap='jet')

# Adjust aspect ratio
ax.set_box_aspect([1, ny/nx, 0.3])  # [x, y, z] aspect ratios

plt.xlabel("x")
plt.ylabel("y")
plt.title("density(x,y)")
plt.show()

In [ ]:
# density(x,y) - surface plot - plotly

import plotly.graph_objects as go
import numpy as np

nx, ny = Dens.shape

fig = go.Figure(go.Surface(z=Dens.T, colorscale='Jet'))

fig.update_layout(
    title='density(x,y)', autosize=False,
    width=1000, height=700,
    scene = {
        'camera_eye': {"x": 0, "y": -1, "z": 0.5},
        "aspectratio": {"x": 1, "y": ny/nx, "z": 0.5}
    })

fig.show()

In [ ]:
# flow vector V(x,y) plot

import numpy as np
import matplotlib.pyplot as plt

# vector field plot
fig = plt.figure(figsize=(12, 8))
ax1 = fig.add_subplot(111)

nx, ny = U.shape
x = np.arange(0, ny)
y = np.arange(0, nx)
Y, X = np.meshgrid(x, y)

scale_factor = 50000
ax1.quiver(X, Y, U, V, scale=scale_factor )
plt.title("flow vector (vx,vy)")
plt.show()

In [ ]:
# flow vector V(x,y) plot - plotly cone plot

import numpy as np
import plotly.graph_objects as go

# prepare numpy data for cone 3d-plot
nx, ny = U.shape
y = np.arange(0, nx)
x = np.arange(0, ny)
Y, X = np.meshgrid(x, y)
Xf = X.flatten()
Yf = Y.flatten()
Zf = Dens.flatten()
Uf = U.flatten()
Vf = V.flatten()
Wf = np.full_like(Uf,0)

# cone plot
fig = go.Figure(data=go.Cone(
    x=Xf, y=Yf, z=Zf, u=Uf, v=Vf, w=Wf,
    colorscale='Jet', sizemode="absolute", sizeref=1000,
    ))

fig.update_layout(
    title='x,y:flow(x,y) z:density(x,y), color:|flow(x,y)|', autosize=False,
    width=1200, height=600,
    scene = {
        'camera_eye': {"x": 0, "y": -1, "z": 0.5},
        "aspectratio": {"x": 1, "y": ny/nx, "z": 0.3}
    })

fig.show()